In [92]:
# configuration
property_label = "P19"
# should be "subjects" or "objects"
orientation = "objects"

In [93]:
import csv
import statistics

node_subject_props = {}
node_object_props = {}
dis_class = set()
tsv_file = open(f'../output/instance_prediction_datasets/{property_label}.{orientation}.1250.tsv',encoding="utf-8")
read_tsv = csv.reader(tsv_file, delimiter="\t")
next(read_tsv)
for line in read_tsv:
    dis_class.add(line[0])
    if line[5] == "subject":
        if line[0] in node_subject_props:
            node_subject_props[line[0]].append(line[2])
        else:
            node_subject_props[line[0]] = [line[2]]
    elif line[5] == "object":
        if line[0] in node_object_props:
            node_object_props[line[0]].append(line[2])
        else:
            node_object_props[line[0]] = [line[2]]
print("distinct classes: ", len(dis_class))

print("making predictions...")
# assign all classes the same guess
# guesses = {}
# for elem in dis_class:
#     guesses[elem] = "Q5"
guesses = predictWithConstraints(dis_class, node_subject_props, node_object_props)

print("retrieving correct answers...")
correct_answers = getCorrectAnswers(list(dis_class))

print("retrieving scores...")
scores = []
for elem in guesses:
    this_score = score(guesses[elem], correct_answers[elem])
    scores.append(this_score)
print("final avg. score: ", statistics.mean(scores))

distinct classes:  1260
making predictions...
retrieving correct answers...
retrieving scores...
final avg. score:  0.17777225396825394


In [2]:
# score P31 datasets
import csv

direct_subclasses = {}
direct_superclasses = {}

tsv_file = open('../derived.P279.tsv/derived.P279.tsv',encoding="utf-8")
read_tsv = csv.reader(tsv_file, delimiter="\t")
next(read_tsv)
for line in read_tsv:
    if line[1] not in direct_superclasses:
        direct_superclasses[line[1]] = [line[3]]
    else:
        direct_superclasses[line[1]].append(line[3])
    if line[3] not in direct_subclasses:
        direct_subclasses[line[3]] = [line[1]]
    else:
        direct_subclasses[line[3]].append(line[1])

In [26]:
import mmap

def getCorrectAnswers(candidates):
    answers = {}
#     with open('../derived.P31.tsv/derived.P31.tsv',encoding="utf-8") as f:
#         s = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
#         while len(candidates) != 0:
#             line = s.readline()
#             for candidate in candidates:
#                 if str.encode("\t"+candidate+"\t") in line:
#                     str1 = line.decode("utf-8")
#                     split = str1.split("\t")
#                     answer = split[-1].split("\n")[0]
#                     answers[candidate] = answer
#                     candidates.remove(candidate)
    tsv_file = open(f'../output/allConstraintsAnalysis_Final/instances/instances.{orientation}.{property_label}.tsv', encoding="utf-8")
    read_tsv = csv.reader(tsv_file, delimiter="\t")
    for line in read_tsv:
        answers[line[0]] = line[1]
    return answers

In [5]:
def score (guess, correct):
    if guess == correct: 
        return 1
    else:
        subclasses = []
        superclasses = []
        if guess in direct_subclasses:
            subclasses = direct_subclasses[guess]
        if guess in direct_superclasses:
            superclasses = direct_superclasses[guess]
        for i in range(1,6):
            new_subclasses = []
            new_superclasses = []
            for subc in subclasses:
                if subc == correct:
                    if i <= 3:
                        return .7**i
                    else:
                        return .8**i
                else:
                    if subc in direct_subclasses:
                        for new_sub in direct_subclasses[subc]:
                            new_subclasses.append(new_sub)
            for supc in superclasses:
                if supc == correct:
                    if i <= 3:
                        return .7**i
                    else:
                        return .8**i
                else:
                    if supc in direct_superclasses:
                        for new_sup in direct_superclasses[supc]:
                            new_superclasses.append(new_sup)
            subclasses = new_subclasses
            superclasses = new_superclasses
    return 0

In [68]:
# load constraint values
typeConstraints = {}
tsv_file = open(f'../data/property_constraints/all_type_constraints.tsv', encoding="utf-8")
read_tsv = csv.reader(tsv_file, delimiter="\t")
next(read_tsv)
for line in read_tsv:
    if line[0].replace("http://www.wikidata.org/entity/","") in typeConstraints:
        typeConstraints[line[0].replace("http://www.wikidata.org/entity/","")].append(line[2].replace("http://www.wikidata.org/entity/",""))
    else:
        typeConstraints[line[0].replace("http://www.wikidata.org/entity/","")] = [line[2].replace("http://www.wikidata.org/entity/","")]

valueTypeConstraints = {}
tsv_file = open(f'../data/property_constraints/all_value_type_constraints.tsv', encoding="utf-8")
read_tsv = csv.reader(tsv_file, delimiter="\t")
next(read_tsv)
for line in read_tsv:
    if line[0].replace("http://www.wikidata.org/entity/","") in valueTypeConstraints:
        valueTypeConstraints[line[0].replace("http://www.wikidata.org/entity/","")].append(line[2].replace("http://www.wikidata.org/entity/",""))
    else:
        valueTypeConstraints[line[0].replace("http://www.wikidata.org/entity/","")] = [line[2].replace("http://www.wikidata.org/entity/","")]

In [74]:
def predictWithConstraints(dis_class, node_subject_props, node_object_props):
    final_guesses = {}
    for node in dis_class:
        all_class_votes = {}
        if node in node_subject_props:
            subject_props = node_subject_props[node]
            for prop in subject_props:
                if prop in typeConstraints:
                    for constraint in typeConstraints[prop]:
                        if constraint in all_class_votes:
                            all_class_votes[constraint] += 1
                        else:
                            all_class_votes[constraint] = 1
        if node in node_object_props:
            object_props = node_object_props[node]
            for prop in object_props:
                if prop in valueTypeConstraints:
                    for constraint in valueTypeConstraints[prop]:
                        if constraint in all_class_votes:
                            all_class_votes[constraint] += 1
                        else:
                            all_class_votes[constraint] = 1
        
        max_vote = ""
        max_score = 0
        for vote in all_class_votes:
            if all_class_votes[vote] > max_score:
                max_vote = vote
                max_score = all_class_votes[vote]
        final_guesses[node] = max_vote
    return final_guesses